# Compare factors delivered by the three methods

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
% matplotlib inline
from sklearn.manifold import TSNE 
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
from factorizer_wrappers import ICA_Factorizer, NMF_Factorizer, PCA_Factorizer
from factorizer_wrappers import example_V, test_example_V, test_Factorizer

In [ ]:
test_example_V()

test_Factorizer(ICA_Factorizer(n_components=4, fun='cube', algorithm='parallel'), atol=0.5) 
test_Factorizer(ICA_Factorizer(n_components=5), atol=0.001) 

test_Factorizer(NMF_Factorizer(n_components=4, max_iter=1000), atol=0.5) 
test_Factorizer(NMF_Factorizer(n_components=5, max_iter=1000), atol=0.1) 

test_Factorizer(PCA_Factorizer(n_components=4), atol=0.5) 
test_Factorizer(PCA_Factorizer(n_components=5), atol=0.001) 

In [ ]:
# Read in expression spreadsheet which has been processed (see end of notebook) to inlcude only protein coding genes
expression_df = pd.read_csv('../Data/HGSOC_Protein_Expression.csv', sep='\t')
expression_df.set_index('GeneENSG', inplace=True)
assert len(expression_df) == 19730   # Only 
assert len(expression_df.columns) == 80
assert expression_df.columns[-1] == 'AOCS_171'
expression_matrix = normalize(np.asarray(expression_df))

print(expression_matrix.shape[0], "genes")
print(expression_matrix.shape[1], "patients")

In [ ]:
# Angle calculation

def calc_angle(v1, v2, show=False):
    
    dotp = np.dot(v1, v2)
    v1_mag = np.sqrt(np.sum(v1 * v1))
    v2_mag = np.sqrt(np.sum(v2 * v2))
    costheta = dotp / (v1_mag * v2_mag)

    angleRad = np.arccos(min(costheta, 1.0))
    angleDeg = angleRad * (180 / np.pi)

    if show:
        print ("v1:\n")
        print (v1)
        print ("\nv2:")
        print (v2)
        print ("\nv1 Mag.:%6.4f" % v1_mag)
        print ("\nv2 Mag.:%6.4f" % v2_mag)
        print ("v1 . v2 = %6.4f" % dotp)
        print(dotp / (m1_mag * m2_mag))
        print ("Angle between v1 and v2 = %5.1f degrees." % angleDeg)
    return angleDeg

calc_angle(v1= np.array([0,0,0,1]), v2=np.array([1,1,1,0]))

In [ ]:
def l2_norm_diff(m1, m2):
    return np.sqrt(np.mean((m1 - m2)**2))

In [ ]:
alist = []
n=10000
for i in range(1000):
    v1 = np.random.randn(n)
    v2 = np.random.randn(n)
    a = calc_angle(v1,v2)
    alist += [a]
    
plt.hist(alist)

In [ ]:
def distance(v1, v2):
    dotp = np.dot(v1, v2)
    v1_mag = np.sqrt(np.sum(v1 * v1))
    v2_mag = np.sqrt(np.sum(v2 * v2))
    costheta = dotp / (v1_mag * v2_mag)
    
    angleRad = np.arccos(abs(min(costheta, 1.0)))
    angleDeg = angleRad * (180 / np.pi)
    return angleDeg

(distance(np.array([1,0,0,1]), np.array([1,0,0,1])), 
 distance(np.array([1,0,0,1]), np.array([0,0,0,1])),
 distance(np.array([1,0,0,0]), np.array([0,0,0,1])))

In [ ]:
# Run ICA multiple times and plot all metagenes in a t-SNE

def run(V, facto_class, n_components, n_repeats):
    pickle_fname = "../Cache/FactorizerExpt/%s_%d_%d.pkl" % (facto_class.__name__, n_components, n_repeats)
    print(pickle_fname)
    metagene_list = []
    for i in range(n_repeats):
        facto = facto_class(n_components=n_components, max_iter=5000, random_state=np.random.randint(10000))
        facto.fit(V)
        metagene_list += [facto.get_W()]
        print('\r%d/%d' % (i+1, n_repeats), end='')
    print()
    with open(pickle_fname, 'wb') as f:
        pickle.dump(metagene_list, f)
        
run(expression_matrix, ICA_Factorizer, 4, 2)

In [ ]:
 if True:
    for nc in range(3, 4):
        run(expression_matrix, NMF_Factorizer, nc, 50)
        run(expression_matrix, ICA_Factorizer, nc, 50)
        print()

    print("All Done.")
    

In [ ]:
def combined_factors_scatter(n_components, n_repeats):
    
    # Read back the pickle files containing multiple runs. One file for each n_components for each
    # of NMF and ICA
    
    dummy_facto = NMF_Factorizer()
    pickle_fname = "../Cache/FactorizerExpt/%s_%d_%d.pkl" % (type(dummy_facto).__name__, n_components, n_repeats)
    # print(pickle_fname)
    with open(pickle_fname, 'rb') as f:
        nmf_metagene_list = pickle.load(f)

    dummy_facto = ICA_Factorizer()
    pickle_fname = "../Cache/FactorizerExpt/%s_%d_%d.pkl" % (type(dummy_facto).__name__, n_components, n_repeats)
    with open(pickle_fname, 'rb') as f:
        ica_metagene_list = pickle.load(f)
        
    # Add result of PCA analysis for same number of components
    pca_facto = PCA_Factorizer(n_components=n_components)
    pca_facto.fit(expression_matrix)
    pca_metagenes = pca_facto.get_W()
    
    stacked_metagenes = np.hstack(nmf_metagene_list + ica_metagene_list + [pca_metagenes]).T
    
    # For ICA at least, we see double the expected number of components, due to the arbitrary direction of the vector
    # So flip them into the same overall direction
    flipped_metagenes = [g if sum(g[:10])>0 else -g for g in stacked_metagenes[:]]

    # Reduce to a managable number of dimensions before passing to t-SNE
    pca = PCA(n_components=50)
    tsne = TSNE(n_components=2, init='pca', n_jobs=7)
    Y = tsne.fit_transform(pca.fit_transform(flipped_metagenes))
    Y.shape

    
    # Plot the t-SNE projections in two halves so that NMF and ICA show in different colours
    assert Y.shape[0] == 2 * n_components * n_repeats + n_components
    # Start indices of the components in Y
    nmf_Y = Y[0:n_components * n_repeats, :]
    ica_Y = Y[n_components * n_repeats: 2 * n_components * n_repeats, :]
    pca_Y = Y[2 * n_components * n_repeats:, :]
    
    plt.scatter(nmf_Y[:,0], nmf_Y[:,1], s=3, label='NMF')
    plt.scatter(ica_Y[:,0], ica_Y[:,1], s=3, label='ICA')
    plt.scatter(pca_Y[:,0], pca_Y[:,1], s=50, marker = '+', label='PCA')
    
    plt.xlabel("t-SNE dimension 1")
    plt.ylabel("t-SNE dimension 2")
    ax = plt.gca()
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    
    
    plt.legend()
    plt.title("Components: %d" % n_components)
    
plt.figure(figsize=(4,4))
combined_factors_scatter(3, 50)
plt.suptitle("Just testing...")

In [ ]:
def plot_multiple_combined_factors_scatter(start_comp, end_comp):
    plt.figure(figsize=(16,20))
    # plt.figure(figsize=(8,8))

    n_repeats = 50
    for nc in range(start_comp, end_comp):
        print('.', end='')
        plt.subplot(4,3,nc-start_comp+1)
        combined_factors_scatter(nc, n_repeats)
    plt.suptitle("t-SNE clustering for %d repeats of NMF and ICA", size=14)
    plt.show()

In [ ]:
plot_multiple_combined_factors_scatter(3, 15)
plot_multiple_combined_factors_scatter(16, 28)

In [ ]:
# Investigate k-means to identify clusters

n_components = 5
n_repeats = 50
dummy_facto = NMF_Factorizer()
pickle_fname = "../Cache/FactorizerExpt/%s_%d_%d.pkl" % (type(dummy_facto).__name__, n_components, n_repeats)
# print(pickle_fname)
with open(pickle_fname, 'rb') as f:
    metagene_list = pickle.load(f)
stacked_metagenes = np.hstack(metagene_list + metagene_list).T
flipped_metagenes = [g if sum(g[:10])>0 else -g for g in stacked_metagenes[:]]
        
pca = PCA(n_components=20)
kmeans = KMeans(n_clusters=n_components, random_state=0).fit(pca.fit_transform(flipped_metagenes))
# kmeans = KMeans(n_clusters=N_COMPONENTS, random_state=0).fit(Ya)
for c in range(n_components):
    print(kmeans.labels_[range(c,n_components*n_repeats,n_components)])

In [ ]:
r = pca.inverse_transform(kmeans.cluster_centers_)
r.shape

In [ ]:
# Calculate angles between components
a = r
n = a.shape[0]
angle_matrix=np.zeros((n,n))
for i1 in range(n):
    for i2 in range(n):
        v1, v2 = a[i1,:], a[i2,:]
        angle_matrix[i1, i2] = calc_angle(v1, v2)

for i1 in range(n):
    print("%2d: " % i1, end="")
    for i2 in range(n):
        if (i2 <= i1):
            print("%6.0f°" % angle_matrix[i1, i2], end="")
    print()
        
        